In [62]:
import pandas as pd
import numpy as np
import heapq
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction import DictVectorizer
from bottleneck import argpartition, partition

from math import ceil
from sklearn.neighbors import LSHForest


In [84]:
class Rec():
    
    def __init__(self):
        self.anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
        self.anime_id = self.anime_tag_vector['anime_id'].as_matrix().tolist()
        self.anime_rating = self.anime_tag_vector['rating']
        del self.anime_tag_vector['anime_id']
        del self.anime_tag_vector['rating']
        self.anime_tag_vector = self.anime_tag_vector.as_matrix()
        
        # user instance: calculated rating to each tag
        
        #self.instance = pd.read_csv('user_instances.csv')
        #self.instance = self.instance.as_matrix()[1:]
        #self.instance = [ row[1:] for row in self.instance ]
        
        self.instance = pd.read_csv('user_instances.csv')
        del self.instance['user_id']
        self.instance = self.instance[1:].as_matrix()
        
        # user actual rating to each watched anime
        self.rate = pd.read_csv('rating.csv')
        self.rate = self.rate.as_matrix().tolist()
        
        self.c = 50
        self.kmeans = cluster.KMeans(n_clusters=self.c, max_iter=300, init='k-means++',n_init=10, verbose=True, n_jobs=-1).fit(self.instance)
        self.clus = self.kmeans.predict(self.instance)
        
        # user watched list
        self.watch = np.array([[] for _ in self.instance])
        for r in self.rate:
            np.append(self.watch[r[0]-1], r[1])
            #self.watch[r[0]-1].append(r[1])

        
        self.animes = pd.read_csv("anime.csv")
        tags = []
        for genres in animes["genre"].fillna(""):
            for genre in str.split(genres, ", "):
                tags.append(genre)
        tags = sorted(list(set(tags))[1:]) # Remove ''
        v = DictVectorizer(sparse="False")
        
        self.genre = v.fit_transform(animes["genre"].fillna(", ".join(tags)).apply(lambda x: {i: 1 for i in str.split(x, ", ")}))
        self.genre = np.array(self.genre.todense())
        
        self.inverse_anime_id = {}
        for index, anime in np.ndenumerate(self.anime_id):
            self.inverse_anime_id[anime] = index[0]
        
        
        self.anime_candidates = []
        
        for i in range(self.c):
            users_in_clus = np.where(self.clus==i)[0] # 'user' should be 'anime' instead

            #X_train = self.instance[users_in_clus]
            #X_test = kmeans.cluster_centers_[i]
            lshf = LSHForest(random_state=42)
            lshf.fit(self.instance[users_in_clus])  # X_train
            LSHForest(min_hash_match=4, n_candidates=50, n_estimators=10,
                          n_neighbors=5, radius=1.0, radius_cutoff_ratio=0.9,
                          random_state=42)
            num_representatives = ceil(len(users_in_clus)/80)
            distances, indices = lshf.kneighbors([self.kmeans.cluster_centers_[i]], n_neighbors=num_representatives) #X_test
            self.anime_candidates.append(list(set([w for i in indices for w in self.watch[i] if w in self.anime_id])))
            ### !!!!list(set([]))

            #print(indices)
            #print()
            
    def get_ranked_ids(self, datadata, k):
        
        # generate user instance from data
        avg_rating = 0
        watch_times = 0
        rating_times = 0
        overall_rating = 0
        total_rating_times = 0
        data = dict(datadata)
        for r in data.values():
            watch_times += 1
            if r != -1:
                avg_rating += r
                rating_times += 1
                overall_rating += r
                total_rating_times += 1
        
        np.seterr(divide='ignore', invalid='ignore')
        avg_rating = np.nan_to_num(avg_rating / rating_times)
        
        avg_overall_rating = overall_rating / total_rating_times
        
        
        
        
        
        ratings = np.zeros(43)
        user_tags = np.zeros(43)
        for anime_id, r in zip(data.keys(), data.values()):
            if anime_id not in self.anime_id:
                continue
            user_tags += self.genre[self.inverse_anime_id[anime_id]]
            if rating_times == 0:
                ratings += self.genre[self.inverse_anime_id[anime_id]] * 5
            
            elif r == -1:
                ratings += self.genre[self.inverse_anime_id[anime_id]] * (10 - avg_rating)
            else:
                ratings += self.genre[self.inverse_anime_id[anime_id]] * r
        
        #user_instance = np.nan_to_num(ratings / user_tags)
        
        #ins = user_instance
        ins = np.nan_to_num(ratings / user_tags)
        
    #    watch = []
     #   for r in data.keys():
     #       watch.append(r)
        
    #    ins_watch = watch

        # predict!!!!
        pred = self.kmeans.predict([ins])[0]
        #index = [i for i, x in enumerate(self.clus.tolist()) if x == pred]
        
        #anime_candidates = [w for i in index for w in self.watch[i] if w in self.anime_id] # to set then to list
        #anime_candidates = set(anime_candidates) - set(ins_watch)
        
        #return heapq.nlargest(k,anime_candidates, key=lambda x: self.anime_rating[self.anime_id.index(x)])
        
        
        #####
        
        argpartition(self.anime_candidates[pred], k)
        
        p = argpartition(self.anime_candidates, len(self.anime_candidates)-k)[len(self.anime_candidates)-k:]
        return sorted(p, key=lambda x: ##[x])[::-1]
        
        
        b = [anime_id.index(a) for a in self.anime_candidates]
        r = anime_rating[b]
        p = argpartition(r, len(r)-k)[len(r)-k:]
        sorted(p, key=lambda x: r[x])[::-1]
        #ans = [anime_id[pp] for pp in p[:2]]
        #ans
        p


In [83]:
rec = Rec()


In [78]:
rate = pd.read_csv('rating.csv')
rate = rate.as_matrix().tolist()

In [79]:
rr = [r for r in rate if r[0]==11]
data = []
for rrr in rr:
    data.append((rrr[1],rrr[2]))
data

[(30, 5),
 (43, 7),
 (44, 7),
 (54, -1),
 (59, 7),
 (61, 4),
 (104, 9),
 (120, 6),
 (121, 7),
 (122, 6),
 (143, 7),
 (149, 5),
 (164, 8),
 (177, 7),
 (199, 10),
 (202, 9),
 (232, 6),
 (243, 4),
 (249, 9),
 (270, -1),
 (338, 10),
 (357, 7),
 (416, 10),
 (430, 6),
 (431, 8),
 (440, 10),
 (441, 9),
 (449, 6),
 (450, 7),
 (451, 6),
 (452, 9),
 (497, 7),
 (512, 5),
 (513, 9),
 (522, 7),
 (523, 8),
 (528, 7),
 (530, 7),
 (531, 7),
 (532, 7),
 (543, 6),
 (565, 6),
 (572, 7),
 (578, 10),
 (589, 9),
 (597, 7),
 (687, 4),
 (723, 7),
 (740, 7),
 (759, 8),
 (790, 8),
 (853, 7),
 (855, 6),
 (879, 7),
 (934, 8),
 (996, 7),
 (997, 7),
 (1002, 7),
 (1032, 8),
 (1060, 8),
 (1117, 9),
 (1118, 8),
 (1239, 7),
 (1240, 7),
 (1709, 6),
 (1889, 8),
 (2167, 7),
 (2313, 10),
 (2685, 6),
 (2847, 6),
 (2890, 6),
 (3577, 6),
 (3652, 6),
 (3702, 9),
 (4059, 7),
 (4181, 8),
 (4896, 6),
 (4898, 6),
 (5030, 6),
 (5060, 7),
 (5114, 8),
 (6164, 6),
 (6280, 7),
 (6351, 9),
 (6707, 4),
 (6747, 7),
 (6811, 8),
 (7593, 8),

In [81]:
rec.get_ranked_ids(data, 5)

ValueError: operands could not be broadcast together with shapes (43,) (44,) (43,) 

In [42]:
anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
anime_id = anime_tag_vector['anime_id'].as_matrix().tolist()
anime_rating = anime_tag_vector['rating']
some_candidates = [32935, 11061, 15417]
b = [anime_id.index(a) for a in some_candidates]
r = anime_rating[b]
p = argpartition(r, 3-2)
#ans = [anime_id[pp] for pp in p[:2]]
#ans
p

array([2, 1, 0])

In [18]:
r

array([ 9.15,  9.13,  9.11])

NameError: name 'clus' is not defined

In [40]:
some_candidates[1:]

[11061, 15417]

In [50]:
test_array = [1,5,4]
partition(test_array, 7)

ValueError: `n` (=7) must be between 0 and 2, inclusive.

In [105]:
r_ = np.hstack((anime_id, anime_rating))
b = [anime_id.index(a) for a in self.anime_candidates]
r = anime_rating[b]
p = argpartition(r, len(r)-k)[len(r)-k:]
sorted(p, key=lambda x: r[x])[::-1]
#ans = [anime_id[pp] for pp in p[:2]]

NameError: name 'self' is not defined

In [106]:
can = [14,11,13]
id = [10,11,12,13,14]
ra = np.array([5, 8, 1, 2, 9])
r_ = np.hstack((id, ra))

In [92]:
b = [id.index(a) for a in can]
b

[4, 1, 3]

In [93]:
r = ra[b]
r

array([9, 8, 2])

In [97]:
k = 2
p = partition(r, len(r)-k)
p

array([2, 8, 9])

In [104]:
[ for i in p]

ValueError: 2 is not in list